In [9]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
import os
import numpy as np
import json
from collections import Counter
from keras.optimizers import SGD

In [2]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [3]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [12]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = Dense(1024, name='dense', activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

ValueError: `input_shape` must be a tuple of three integers.

In [5]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15
49s - loss: 1.5749 - acc: 0.6681
Epoch 2/15
32s - loss: 0.5904 - acc: 0.8281
Epoch 3/15
32s - loss: 0.4022 - acc: 0.8725
Epoch 4/15
32s - loss: 0.3413 - acc: 0.8950
Epoch 5/15
32s - loss: 0.2783 - acc: 0.9088
Epoch 6/15
32s - loss: 0.2452 - acc: 0.9168
Epoch 7/15
32s - loss: 0.2101 - acc: 0.9283
Epoch 8/15
33s - loss: 0.1836 - acc: 0.9359
Epoch 9/15
32s - loss: 0.1551 - acc: 0.9470
Epoch 10/15
32s - loss: 0.1634 - acc: 0.9478
Epoch 11/15
33s - loss: 0.1246 - acc: 0.9594
Epoch 12/15
33s - loss: 0.1094 - acc: 0.9625
Epoch 13/15
33s - loss: 0.1038 - acc: 0.9654
Epoch 14/15
33s - loss: 0.0856 - acc: 0.9714
Epoch 15/15
33s - loss: 0.0925 - acc: 0.9697


In [6]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=2
)

Epoch 1/10
34s - loss: 0.0207 - acc: 0.9935
Epoch 2/10
34s - loss: 0.0184 - acc: 0.9946
Epoch 3/10
34s - loss: 0.0176 - acc: 0.9955
Epoch 4/10
34s - loss: 0.0166 - acc: 0.9959
Epoch 5/10
34s - loss: 0.0155 - acc: 0.9972
Epoch 6/10
34s - loss: 0.0156 - acc: 0.9972
Epoch 7/10
34s - loss: 0.0149 - acc: 0.9969
Epoch 8/10
34s - loss: 0.0152 - acc: 0.9965
Epoch 9/10
34s - loss: 0.0151 - acc: 0.9963
Epoch 10/10
35s - loss: 0.0146 - acc: 0.9976


In [11]:
json.dump(idx_to_labels, open('zoo/09.3 pet_labels.json', 'w'))
model.save('zoo/09.3 retrained pet recognizer.h5')
